In [1]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from Cogniezer.constants import *
from Cogniezer.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
from Cogniezer.utils.common import get_size
from Cogniezer.logging import logger

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file)
            logger.info(f"Downloaded file {filename} with headers :\n{headers}")
        else:
            logger.info(f"File already exist of size {get_size(Path(self.config.local_data_file))}")

    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        if not os.path.exists(unzip_path):
            os.makedirs(unzip_path)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    logger.exception(e)
    raise e